In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pybaseball import statcast, cache
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import joblib
import math
from pybaseball import statcast
import scipy.stats as stats
#from catboost import Pool
#import optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score
import sqlite3
cache.enable()

In [2]:
data = pd.read_csv('savant_data.csv')

In [3]:
data['velocity'].mean()

np.float64(87.4898731408574)

In [4]:
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'pitch_percent'
# - 'swing_miss_percent'
# - 'arm_angle'
# - 'xbadiff'
# - 'xobp'
# - 'xslg'
# - 'pitcher_run_value_per_100'
# - 'batter_run_value_per_100'

# We will also select 'swing_miss_percent' as our initial target variable

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'pitch_percent', 'swing_miss_percent', 'arm_angle', 
    'xbadiff', 'xobp', 'xslg', 'pitcher_run_value_per_100', 'batter_run_value_per_100'
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['swing_miss_percent']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse


np.float64(0.005906508196721224)

In [5]:
comparison = pd.DataFrame({
    'Actual': y_test,
    'Predicted': y_pred
})

print(comparison)

      Actual  Predicted
538     40.0     40.003
1822    22.6     22.599
534     40.5     40.498
2119    18.5     18.500
561     30.6     30.599
...      ...        ...
744     38.8     38.798
733     43.0     42.892
2647    26.0     26.000
4149     9.5      9.505
3819    16.7     16.700

[915 rows x 2 columns]


In [6]:
#mean_whiff = data['swing_miss_percent'].mean()
#data['Stuff+'] = (data['swing_miss_percent'] / mean_whiff) * 100

# Normalizing the columns
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100
data['xwOBA_norm'] = (1 - (data['xwoba'] / data['xwoba'].max())) * 100 # Inverse
data['RunExp_norm'] = (1 - (data['run_exp'] / data['run_exp'].max())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = .3, .2, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()


,swing_miss_percent,xobp,pitcher_run_value_per_100,Stuff+
0,44.9,0.234,1.495762,153.506925
1,25.7,0.311,1.273859,151.563868
2,39.5,0.261,1.302039,139.285099
3,26.0,0.285,1.117130,135.211261
4,28.9,0.281,1.143414,132.471600


In [7]:
# Sort the DataFrame by the 'Stuff+' column in descending order (higher Stuff+ is better)
features_sorted = data.sort_values(by='Stuff+', ascending=False)
features_sorted.to_csv('savant_data.csv', index=False)

In [8]:
'''
data['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100
data['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse
data['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse

# Assigning weights
w1, w2, w3 = .3, .2, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()
'''

"\ndata['SwingMiss_norm'] = (data['swing_miss_percent'] / data['swing_miss_percent'].mean()) * 100\ndata['xwOBA_norm'] = ((data['xwoba'] / data['xwoba'].mean())) * 100 # Inverse\ndata['RunExp_norm'] = ((data['run_exp'] / data['run_exp'].mean())) * 100  # Inverse\n\n# Assigning weights\nw1, w2, w3 = .3, .2, .5\n\n# Calculating Stuff+\ndata['Stuff+'] = (\n    w1 * data['SwingMiss_norm'] +\n    w2 * data['xwOBA_norm'] +\n    w3 * data['RunExp_norm']\n)\n\n# Display the first few rows of the Stuff+ calculation\ndata[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()\n"

In [11]:
print(data['xwoba'].min())
print(data['xwoba'].max())
print(data['xwoba'].mean())
print(data['pitcher_run_exp'].min())
print(data['pitcher_run_exp'].max())
print(data['pitcher_run_exp'].mean())


0.0
1.989
0.32474518810148734
-53.1
79.3
0.2558180227471567


In [13]:
# different normailzation so all are around 60-150, average at 100
data['pre_new_all_+'] = data['pitcher_run_exp'] - data['pitcher_run_exp'].min()
print(data['pre_new_all_+'].mean())

53.35581802274715


In [104]:
std_dev_pre = data['pitcher_run_exp'].std()
std_dev_xwoba = data['xwoba'].std()
std_dev_swm = data['swing_miss_percent'].std()

# try with standard dev
data['SwingMiss_norm'] = ((data['swing_miss_percent'] - data['swing_miss_percent'].mean()) / std_dev_swm) * 10
data['xwOBA_norm'] = (((data['xwoba'].mean() - data['xwoba'])) / std_dev_xwoba) * 10
data['RunExp_norm'] = ((data['pitcher_run_exp'] - data['pitcher_run_exp'].mean()) / std_dev_pre) * 5

# Assigning weights
w1, w2, w3 = .2, .3, .5

# Calculating Stuff+
data['Stuff+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['xwOBA_norm'] +
    w3 * data['RunExp_norm']
)

data['Stuff+'] = (data['Stuff+'] + 100)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'xobp', 'pitcher_run_value_per_100', 'Stuff+']].head()

,swing_miss_percent,xobp,pitcher_run_value_per_100,Stuff+
0,44.9,0.234,1.495762,127.548710
1,25.7,0.311,1.273859,124.233678
2,39.5,0.261,1.302039,122.356066
3,26.0,0.285,1.117130,119.100327
4,28.9,0.281,1.143414,118.802649


In [105]:
features_sorted = data.sort_values(by='Stuff+', ascending=False)
print(data['Stuff+'].mean())
features_sorted.to_csv('savant_data.csv', index=False)

100.0


In [106]:
filtered_data = data[data['pitches'] >= 200]
filtered_data = filtered_data.sort_values(by='Stuff+', ascending=False)
filtered_data.to_csv('STUFF+Initial.csv', index=False)

In [107]:
print(filtered_data[filtered_data['pitch_type'] == 'FC'].head())

    pitches  player_id         player_name  total_pitches pitch_type  \
1      6224     669203      Burnes, Corbin          11943         FC   
34     1278     621237      Alvarado, José           3720         FC   
14     2802     661403     Clase, Emmanuel           4080         FC   
90      318     596112  Stephenson, Robert           2444         FC   
31     2028     502085    Robertson, David           3546         FC   

    pitch_percent     ba    iso  babip    slg  ...  xobpdiff  xslgdiff  \
1            52.1  0.227  0.115  0.278  0.341  ...    -0.012    -0.022   
34           34.4  0.136  0.063  0.327  0.199  ...     0.018     0.035   
14           68.7  0.212  0.071  0.259  0.283  ...    -0.022    -0.040   
90           13.0  0.101  0.152  0.143  0.253  ...    -0.020     0.008   
31           57.2  0.196  0.084  0.283  0.280  ...    -0.005    -0.041   

    wobadiff  swing_miss_percent  arm_angle      Stuff+  SwingMiss_norm  \
1     -0.015                25.7       41.2  12

In [112]:
std_dev_ba = data['ba'].std()
std_dev_ls = data['launch_speed'].std()
std_dev_swm = data['swing_miss_percent'].std()

# try with standard dev
data['SwingMiss_norm'] = ((data['swing_miss_percent'] - data['swing_miss_percent'].mean()) / std_dev_swm) * 10
data['ls_norm'] = ((data['launch_speed'].mean() - data['launch_speed']) / std_dev_ls) * 10
data['ba_norm'] = ((data['ba'].mean() - data['ba']) / std_dev_ba) * 5

# Assigning weights
w1, w2, w3 = 1/3, 1/3, 1/3

# Calculating Stuff+
data['Unhittable+'] = (
    w1 * data['SwingMiss_norm'] +
    w2 * data['ls_norm'] +
    w3 * data['ba_norm']
)

data['Unhittable+'] = (data['Unhittable+'] + 100)

# Display the first few rows of the Stuff+ calculation
data[['swing_miss_percent', 'ba', 'launch_speed', 'Unhittable+']].head()

,swing_miss_percent,ba,launch_speed,Unhittable+
0,44.9,0.169,86.3,108.181238
1,25.7,0.227,87.8,100.603362
2,39.5,0.169,85.6,107.052060
3,26.0,0.211,91.3,98.411275
4,28.9,0.211,89.0,100.917042


In [114]:
filtered_data_unhit = data[data['pitches'] >= 200]
filtered_data_unhit = filtered_data_unhit.sort_values(by='Unhittable+', ascending=False)
filtered_data_unhit.to_csv('Unhittable+Initial.csv', index=False)
data['Unhittable+'].mean()

np.float64(100.0)

In [ ]:
# Stuff Model!
# First, let's select the key features for the "Stuff+" model as discussed:
# - 'pitch_type'
# - 'velocity'
# - 'pfx_z'
# - 'pfx_x'
# - 'release_extension'
# - 'xobp'
# - 'xslg'
# - 'pitcher_run_value_per_100'
# - 'batter_run_value_per_100'


avg_spin_rateR = all_four_seamsR['release_spin_rate'].mean()
avg_x_posR = all_four_seamsR['release_pos_x'].mean()
avg_z_posR = all_four_seamsR['release_pos_z'].mean()
FF_avgR = [avg_veloR, avg_vertR, avg_horizR, avg_extensionR, avg_spin_rateR, avg_x_posR, avg_z_posR]
print(FF_avgR)

# We will also select 'swing_miss_percent' as our initial target variable

# Selecting the relevant features
selected_columns = [
    'pitch_type', 'velocity', 'pfx_z', 'pfx_x', 'release_extension'
]

data = data.dropna(subset=selected_columns)

# Extracting the features and target variable
features = data[selected_columns]
target = data['swing_miss_percent']

# One-hot encoding the categorical 'pitch_type' column to handle it in the model
features_encoded = pd.get_dummies(features, columns=['pitch_type'], drop_first=True)

# Splitting the data into training and testing sets for modeling
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features_encoded, target, test_size=0.2, random_state=42)

# We will now train an initial regression model. I'll start with a Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialize the model
rf_model = RandomForestRegressor(random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Calculate the mean squared error for performance evaluation
mse = mean_squared_error(y_test, y_pred)
mse
